Задача тематическое моделирование

продолжаем исследование датасета с твитами

Скачиваем датасет (источник): положительные, отрицательные.

или можно через ноутбук

!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv


что надо сделать
1. объединить в одну выборку (это только для твитов), для роспотребнадзора сформировать датасет из вопросов
2. провести исследование и выявить тематики о которых говорят в твитах (для твитов), а для роспотребнадзора так же выявить тематики о которых люди пишут проанализировать
3. сделать визуализацию кластеров тематик
4. проинтерпритировать получившиеся тематики

**I think that most tweets are just emotional reactions to smth, aka BS. They are usually without any coherent contence, so it's very interesting to see the model's attempt to sqweeze anything worthy out of them**

In [1]:
!apt-get --yes update
!apt-get --yes install git make cmake build-essential libboost-all-dev
!apt-get --yes install python-numpy python-pandas python-scipy
!wget https://bootstrap.pypa.io/get-pip.py
!python get-pip.py
!pip install protobuf tqdm wheel
!git clone --branch=stable https://github.com/bigartm/bigartm.git

Hit:1 http://archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Get:4 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:6 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Fetched 336 kB in 3s (99.9 kB/s)
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
make is already the newest version (4.2.1-1

import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path) 

In [2]:
%cd bigartm

/content/bigartm


In [3]:
!mkdir build

mkdir: cannot create directory ‘build’: File exists


In [4]:
%cd build

/content/bigartm/build


In [5]:
!cmake ..

-- Build type: Release
-- Found Boost: /usr/lib/x86_64-linux-gnu/cmake/Boost-1.71.0/BoostConfig.cmake (found version "1.71.0")  
-- Found Boost: /usr/lib/x86_64-linux-gnu/cmake/Boost-1.71.0/BoostConfig.cmake (found version "1.71.0") found components: thread program_options date_time filesystem iostreams system 
CMake Deprecation Warning at 3rdparty/CMakeLists.txt:1 (cmake_minimum_required):
  Compatibility with CMake < 2.8.12 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value or use a ...<max> suffix to tell
  CMake that the project does not need compatibility with older versions.


CMake Warning (dev) at 3rdparty/CMakeLists.txt:13 (set):
  implicitly converting '' to 'STRING' type.
This warning is for project developers.  Use -Wno-dev to suppress it.

CMake Warning (dev) at 3rdparty/CMakeLists.txt:19 (set):
  implicitly converting '' to 'STRING' type.
This warning is for project developers.  Use -Wno-dev to suppress it.

CMake Warning (dev) at

In [6]:
!make

[ 28%] Built target libprotobuf
[ 61%] Built target libprotoc
[ 63%] Built target protoc
[ 63%] Built target proto_generation
[ 64%] Built target internals_proto
[ 65%] Built target gflags_static
[ 68%] Built target glog
[ 69%] Built target messages_proto
[ 89%] Built target artm-static
[ 90%] Built target artm
[ 97%] Built target artm_tests
[ 98%] Built target bigartm
[ 98%] Building python package bigartm
/usr/local/lib/python3.10/dist-packages/setuptools/__init__.py:84: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
!!

        ********************************************************************************
        Requirements should be satisfied by a PEP 517 installer.
        If you are using pip, you can try `pip install --use-pep517`.
        ********************************************************************************

!!
  dist.fetch_build_eggs(dist.setup_requires)
running build
running build_py
[ 98%] Built target python_bigartm_build
[100

In [7]:
!make install

[ 28%] Built target libprotobuf
[ 61%] Built target libprotoc
[ 63%] Built target protoc
[ 63%] Built target proto_generation
[ 64%] Built target internals_proto
[ 65%] Built target gflags_static
[ 68%] Built target glog
[ 69%] Built target messages_proto
[ 89%] Built target artm-static
[ 90%] Built target artm
[ 97%] Built target artm_tests
[ 98%] Built target bigartm
[ 98%] Building python package bigartm
/usr/local/lib/python3.10/dist-packages/setuptools/__init__.py:84: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
!!

        ********************************************************************************
        Requirements should be satisfied by a PEP 517 installer.
        If you are using pip, you can try `pip install --use-pep517`.
        ********************************************************************************

!!
  dist.fetch_build_eggs(dist.setup_requires)
running build
running build_py
[ 98%] Built target python_bigartm_build
[100

In [8]:
!pip install python/bigartm*.whl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./python/bigartm-0.10.0-cp310-cp310-linux_x86_64.whl
bigartm is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.


In [9]:
import numpy as np
import re

In [10]:
from collections import Counter

In [11]:
from google.colab import drive
drive.mount('/content/MyDrive')

Drive already mounted at /content/MyDrive; to attempt to forcibly remount, call drive.mount("/content/MyDrive", force_remount=True).


In [12]:
import artm

In [13]:
artm.version()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'0.10.0'

In [14]:
artm.ARTM(num_topics=10).info

config {
  topic_name: "topic_0"
  topic_name: "topic_1"
  topic_name: "topic_2"
  topic_name: "topic_3"
  topic_name: "topic_4"
  topic_name: "topic_5"
  topic_name: "topic_6"
  topic_name: "topic_7"
  topic_name: "topic_8"
  topic_name: "topic_9"
  score_config {
    name: "^^^ItemsProcessedScore^^^"
    type: ScoreType_ItemsProcessed
    config: ""
  }
  pwt_name: "pwt"
  nwt_name: "nwt"
  num_document_passes: 10
  reuse_theta: false
  cache_theta: false
}
score {
  name: "^^^ItemsProcessedScore^^^"
  type: "N4artm5score14ItemsProcessedE"
}
processor_queue_size: 0
num_processors: 2

In [15]:
import pandas as pd
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

<ipython-input-15-5b7176a9d163>:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


In [16]:
import matplotlib.pyplot as plt
%matplotlib inline

In [17]:
!pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
import contractions
from string import punctuation
from collections import Counter

In [19]:
import pickle
with open('/content/MyDrive/MyDrive/Python/data/Emoji_Dict.p', 'rb') as fp:
    Emoji_Dict = pickle.load(fp)
    Emoji_Dict = {v: k for k, v in Emoji_Dict.items() if ('face' in k or 'hand' in k)}

In [20]:
def convert_to_vw_format(index, row):
    return f'doc{index} {row.text}'

In [21]:
def preprocess_text(texts_list): # texts_list - an iterable of tweets
    prepared_texts = []
    
    for text in texts_list:
        
        # 1) Удалим @user из всех твитов с помощью паттерна "@[\w]*". 
        text = re.sub(r"@[\w]*", " ", text)
        
        # 3) Заменим сокращения с апострофами (пример: ain't, can't) на пробел & 4)Заменим сокращения на их полные формы
        expanded_words = []   
        for word in text.split():
            # using contractions.fix to expand the shortened words
            expanded_words.append(contractions.fix(word))            
        text = ' '.join(expanded_words)
        
        # 5) Заменим эмотиконы (пример: ":)" = "happy") на пробелы 
        # 
        for emot in Emoji_Dict:
            text = re.sub(r'('+emot+')', ' ', text)
        
        # 6) пунктуацию, используя re.sub() и паттерн r'[^\w\s]'
        text = re.sub(r'[^\w\s]', " ", text)
        
           
        # 9) Удалим из текста слова длиной в 2 символa, используя ' '.join([w for w in x.split() if len(w)>2])
        
        text = ' '.join([w for w in text.split() if len(w)>2])
        prepared_texts.append(text)
        
    
    return prepared_texts

In [22]:
# считываем данные и заполняем общий датасет
positive = pd.read_csv("/content/MyDrive/MyDrive/Python/data/positive.csv", sep=';', usecols=[3], names=['text'])
negative = pd.read_csv("/content/MyDrive/MyDrive/Python/data/negative.csv", sep=';', usecols=[3], names=['text'])
df = positive.append(negative)


<ipython-input-22-2ea93172f874>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = positive.append(negative)


In [23]:
df.head(2)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,text
0,"@first_timee хоть я и школота, но поверь, у нас то же самое :D общество профилирующий предмет типа)"
1,"Да, все-таки он немного похож на него. Но мой мальчик все равно лучше:D"


In [24]:
prepared_texts = preprocess_text(df.text)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [25]:
df_prepared = pd.DataFrame(prepared_texts, columns=['text'])
df_prepared = df_prepared.sample(frac=1).reset_index(drop=True)


In [26]:
df_prepared.head(3)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,text
0,ура восстановили стим без каких либо вопросов
1,Посл нед дерьмонастроение сегодня днем было норм была теми людьми кем хорошо сейчас началось опять дерьмо
2,Котик коварный Нужно его наказать


In [27]:
df_prepared['text'].iloc[4]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'боже моя мечта'

In [28]:
with open("/content/MyDrive/MyDrive/Python/data/tweets_wv.wv", "w") as f:
    for index, row in df_prepared.iterrows():
        vw_string = convert_to_vw_format(index, row)
        f.write(vw_string)
        f.write('\n')

In [29]:
batch_vectorizer = artm.BatchVectorizer(data_path='/content/MyDrive/MyDrive/Python/data/tweets_wv.wv', 
                                        data_format='vowpal_wabbit',
                                        target_folder='lenta_batches')

In [30]:
%%time 
lda = artm.LDA(num_topics=10, alpha=0.1, beta=0.001, 
               cache_theta=True,
               num_document_passes=5, 
               dictionary=batch_vectorizer.dictionary)

lda.fit_offline(batch_vectorizer=batch_vectorizer, 
                num_collection_passes=10)

CPU times: user 2min 12s, sys: 1.91 s, total: 2min 14s
Wall time: 1min 31s


In [31]:
lda.perplexity_value

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[222661.25,
 8844.7197265625,
 7371.00390625,
 6597.4296875,
 6209.28662109375,
 5972.47265625,
 5806.52001953125,
 5682.28955078125,
 5586.052734375,
 5509.8466796875]

In [32]:
np.log(lda.perplexity_last_value)

8.614292075931775

In [57]:
top_tokens = lda.get_top_tokens(num_tokens=20)
for i, token_list in enumerate(top_tokens):
     print ('Topic #{0}: {1}'.format(i, token_list))

Topic #0: ['так', 'что', 'будет', 'это', 'есть', 'вот', 'как', 'все', 'буду', 'хорошо', 'теперь', 'меня', 'пока', 'сейчас', 'завтра', 'была', 'где', 'опять', 'раз', 'если']
Topic #1: ['только', 'тебя', 'когда', 'как', 'очень', 'сегодня', 'люблю', 'было', 'уже', 'день', 'завтра', 'всем', 'без', 'они', 'Мне', 'хоть', 'плохо', 'могу', 'тоже', 'вчера']
Topic #2: ['что', 'нет', 'надо', 'день', 'вообще', 'все', 'даже', 'ничего', 'делать', 'меня', 'как', 'уже', 'много', 'без', 'там', 'этого', 'после', 'домой', 'было', 'человек']
Topic #3: ['меня', 'все', 'как', 'его', 'если', 'больше', 'потом', 'тоже', 'тут', 'через', 'там', 'часа', 'два', 'было', 'всегда', 'был', 'друг', 'или', 'чего', 'для']
Topic #4: ['мне', 'http', 'кто', 'его', 'она', 'всегда', 'нравится', 'все', 'мной', 'таки', 'моя', 'уже', 'мой', 'о_О', 'себе', 'нибудь', 'чем', 'Новый', 'время', 'наверное']
Topic #5: ['это', 'что', 'тебе', 'знаю', 'меня', 'http', 'как', 'вот', 'она', 'чем', 'или', 'всех', 'для', 'понимаю', 'кого', 'на

**total bs )))))))))) Trying out to exclude frequente and rare words**

In [34]:
batch_vectorizer.dictionary.filter(min_tf=10, max_tf=5000)

artm.Dictionary(name=dac0a18d-abd0-429f-9867-f16892de0b0d, num_entries=18446)

In [35]:
plsa = artm.ARTM(num_topics=10, 
                 dictionary=batch_vectorizer.dictionary)
plsa.scores.add(artm.PerplexityScore(name='lenta_perplexity_score',
                dictionary=batch_vectorizer.dictionary))

plsa.scores.add(artm.TopTokensScore(name='TopTokensScore', 
                                    num_tokens=6))
plsa.fit_offline(batch_vectorizer=batch_vectorizer, 
                 num_collection_passes=20)

In [36]:
for topic_name in plsa.topic_names:
    print (topic_name + ': ', plsa.score_tracker['TopTokensScore'].last_tokens[topic_name])

topic_0:  ['есть', 'завтра', 'буду', 'сейчас', 'раз', 'где']
topic_1:  ['без', 'они', 'Сегодня', 'всем', 'Мне', 'Все']
topic_2:  ['надо', 'вообще', 'ничего', 'делать', 'после', 'много']
topic_3:  ['его', 'если', 'для', 'тут', 'больше', 'был']
topic_4:  ['кто', 'она', 'всегда', 'Это', 'себя', 'себе']
topic_5:  ['знаю', 'или', 'чем', 'такой', 'всех', 'один']
topic_6:  ['даже', 'про', 'блин', 'год', 'плохо', 'нибудь']
topic_7:  ['нас', 'Как', 'люблю', 'почему', 'можно', 'Вот']
topic_8:  ['теперь', 'спасибо', 'дома', 'время', 'ещё', 'вас']
topic_9:  ['там', 'всё', 'могу', 'хорошо', 'мой', 'может']


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**stop_words**

In [42]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [43]:
stop_words = stopwords.words('russian')

In [49]:
def excl_stopwords(texts_list): # texts_list - an iterable of tweets
    prepared_texts = []
    
    for text in texts_list:
        
        text = ' '.join([w for w in text.split() if w not in stop_words])
        prepared_texts.append(text)
    
    return prepared_texts

In [50]:
stopwords_tweets = excl_stopwords(df_prepared.text)
df_stopwords = pd.DataFrame(stopwords_tweets, columns=['text'])

In [51]:
df_stopwords.iloc[145]

text    Серега молодой лет страшно ппц Сын наверно возил
Name: 145, dtype: object

In [52]:
with open("/content/MyDrive/MyDrive/Python/data/tweets_stopwords_wv.wv", "w") as f:
    for index, row in df_stopwords.iterrows():
        vw_string = convert_to_vw_format(index, row)
        f.write(vw_string)
        f.write('\n')

In [53]:
batch_vectorizer_stop = artm.BatchVectorizer(data_path='/content/MyDrive/MyDrive/Python/data/tweets_stopwords_wv.wv', 
                                        data_format='vowpal_wabbit',
                                        target_folder='lenta_batches')

In [58]:
%%time 
lda_stop = artm.LDA(num_topics=10, alpha=0.1, beta=0.001, 
               cache_theta=True,
               num_document_passes=5, 
               dictionary=batch_vectorizer_stop.dictionary)

lda_stop.fit_offline(batch_vectorizer=batch_vectorizer_stop, 
                num_collection_passes=10)

CPU times: user 2min 14s, sys: 2.21 s, total: 2min 17s
Wall time: 1min 36s


In [59]:
lda_stop.perplexity_value

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[220152.5,
 16048.15625,
 12803.2919921875,
 11196.8642578125,
 10428.4267578125,
 9973.8076171875,
 9662.775390625,
 9432.6015625,
 9254.98046875,
 9114.7802734375]

In [61]:
np.log(lda_stop.perplexity_last_value)

9.11765258078134

In [62]:
top_tokens_stop = lda_stop.get_top_tokens(num_tokens=20)
for i, token_list in enumerate(top_tokens_stop):
     print ('Topic #{0}: {1}'.format(i, token_list))

Topic #0: ['буду', 'http', 'пока', 'это', 'смотреть', 'правда', 'скоро', 'Очень', 'надеюсь', 'кстати', 'будут', 'думаю', 'смотрю', 'первый', 'одной', 'боюсь', 'завтра', 'жду', 'снова', 'реально']
Topic #1: ['сегодня', 'очень', 'завтра', 'всем', 'Мне', 'равно', 'утро', 'Завтра', 'день', 'школу', 'вчера', 'ночи', 'мало', 'школе', 'долго', 'ладно', 'скучно', 'Все', 'будем', 'дня']
Topic #2: ['день', 'вообще', 'делать', 'домой', 'ахах', 'никто', 'весь', 'человек', 'Сегодня', 'идти', 'Блин', 'раньше', 'каждый', 'сколько', 'времени', 'сделать', 'настроение', 'рождения', 'сказать', 'уроки']
Topic #3: ['Спасибо', 'часа', 'нужно', 'друг', 'работу', 'поняла', 'давай', 'лет', 'самое', 'столько', 'ахахах', 'назад', 'amp', 'ехать', 'боль', 'дня', 'рано', 'скорее', 'хватает', 'подарок']
Topic #4: ['http', 'Это', 'мной', 'такое', 'таки', 'Только', 'о_О', 'наверное', 'значит', 'Новый', 'жизни', 'жалко', 'пиздец', 'рядом', 'читать', 'написать', 'зима', 'время', 'євромайдан', 'ужасно']
Topic #5: ['это',

**Not much better. Last attempt with word frequency**

In [63]:
batch_vectorizer_stop.dictionary.filter(min_tf=10, max_tf=5000)

artm.Dictionary(name=a378eafc-ad2a-406c-8058-70269cce1b1a, num_entries=18347)

In [66]:
plsa_stop = artm.ARTM(num_topics=10, 
                 dictionary=batch_vectorizer.dictionary)
plsa_stop.scores.add(artm.PerplexityScore(name='lenta_perplexity_score',
                dictionary=batch_vectorizer_stop.dictionary))

plsa_stop.scores.add(artm.TopTokensScore(name='TopTokensScore', 
                                    num_tokens=15))
plsa_stop.fit_offline(batch_vectorizer=batch_vectorizer_stop, 
                 num_collection_passes=20)

In [67]:
for topic_name in plsa_stop.topic_names:
    print (topic_name + ': ', plsa_stop.score_tracker['TopTokensScore'].last_tokens[topic_name])

topic_0:  ['завтра', 'буду', 'пока', 'скоро', 'школу', 'думаю', 'правда', 'снова', 'идти', 'скучаю', 'надеюсь', 'будут', 'кому', 'Очень', 'первый']
topic_1:  ['всем', 'Мне', 'Все', 'дня', 'утро', 'равно', 'Если', 'Завтра', 'сразу', 'ночи', 'мало', 'ладно', 'школе', 'долго', 'будем']
topic_2:  ['вообще', 'делать', 'домой', 'ахах', 'никто', 'человек', 'Блин', 'сделать', 'сильно', 'раньше', 'времени', 'сколько', 'свой', 'каждый', 'сказать']
topic_3:  ['нужно', 'Спасибо', 'лет', 'часа', 'который', 'столько', 'самое', 'друг', 'фото', 'работу', 'ахахах', 'такого', 'человека', 'давай', 'поняла']
topic_4:  ['Это', 'время', 'такое', 'жизни', 'жизнь', 'мной', 'таки', 'Только', 'о_О', 'наверное', 'значит', 'Новый', 'пиздец', 'жалко', 'рядом']
topic_5:  ['знаю', 'нам', 'кого', 'весь', 'одна', 'понимаю', 'жаль', 'кажется', 'Теперь', 'сижу', 'Почему', 'думала', 'улице', 'Когда', 'кем']
topic_6:  ['блин', 'год', 'плохо', 'Так', 'новый', 'ахаха', 'хотела', 'писать', 'минут', 'хочет', 'норм', 'ужасно',

**Q.E.D.**